In [ ]:
############################ Basic function definition for S2S // No modification required ########################################333333
parameter_num = {'mn2t6': 122,'mx2t6': 121,'tp': 228228,'10u': 165,'10v': 166,'ssr':176,'2d':168,'2t': 167}
institutions= {'ecmf': 'ECMWF','babj': 'CMA', 'kwbc': 'NCEP'}
num="1/2/3/4/5/6/7/8/9/10" # Prediction member numbers
# The forecast interval for mn2t6 (minimum 2m temperature 6-hourly), mx2t6 (maximum 2m temperature 6-hourly), and tp (total precipitation) is 6 hours, missing the 00:00 of day 0. 
# Therefore, forecast 47*4-1=187 fields in the future. # Verify that 06:00, 12:00, and 18:00 of 47 days are not missing
step1 = """6/12/18/24/30/36/42/48/54/60/66/72/78/84/90/96/102/108/114/120/126/132/138/144/150/156/162/168
/174/180/186/192/198/204/210/216/222/228/234/240/246/252/258/264/270/276/282/288/294/300/306/312
/318/324/330/336/342/348/354/360/366/372/378/384/390/396/402/408/414/420/426/432/438/444/450/456
/462/468/474/480/486/492/498/504/510/516/522/528/534/540/546/552/558/564/570/576/582/588/594/600
/606/612/618/624/630/636/642/648/654/660/666/672/678/684/690/696/702/708/714/720/726/732/738/744
/750/756/762/768/774/780/786/792/798/804/810/816/822/828/834/840/846/852/858/864/870/876/882/888
/894/900/906/912/918/924/930/936/942/948/954/960/966/972/978/984/990/996/1002/1008/1014/1020/1026
/1032/1038/1044/1050/1056/1062/1068/1074/1080/1086/1092/1098/1104"""

# Interpolation required / Not required
# The forecast interval for tp is 6 hours, cumulative precipitation every 6 hours starting from the 0th time step, but only need to download 00:00 daily
# The forecast interval for ssr (surface solar radiation) is 24 hours
step2="""0/24/48/72/96/120/144/168/192/216/240/264/288/312/336/360/384/408/432/456/480/504/528/552/576
/600/624/648/672/696/720/744/768/792/816/840/864/888/912/936/960/984/1008/1032/1056/1080/1104"""

# No interpolation required
# After 2020-01-02, the forecasts for 10u (10m zonal wind) and 10v (10m meridional wind) were changed to 6-hourly, but only need to download 00:00 daily
step3="""24/48/72/96/120/144/168/192/216/240/264/288/312/336/360/384/408/432/456/480/504/528/552/576
/600/624/648/672/696/720/744/768/792/816/840/864/888/912/936/960/984/1008/1032/1056/1080/1104"""


# Daily averaged
# 2d (2 metre dewpoint temperature) and 2t (2 metre temperature) are daily aggregated
step4 = """0-24/24-48/48-72/72-96/96-120/120-144/144-168/168-192/192-216/216-240/240-264/264-288/288-312/312-336
/336-360/360-384/384-408/408-432/432-456/456-480/480-504/504-528/528-552/552-576/576-600
/600-624/624-648/648-672/672-696/696-720/720-744/744-768/768-792/792-816/816-840/840-864/864-888
/888-912/912-936/936-960/960-984/984-1008/1008-1032/1032-1056/1056-1080/1080-1104"""



import json
import os
import sys
from datetime import datetime, timedelta,date
root_directory = os.getcwd()[0:3]
sys.path.append(root_directory+'SCI\\SCI9_1\\01_code')
sys.path.append(r'C:\ProgramData\anaconda3\Lib\site-packages') 
sys.path.append(r'C:\Users\DELL\.conda\envs\myenv\Lib\site-packages') 
#sys.path.append(r'C:\Users\DELL\.conda\envs\rasterio_env\Lib\site-packages') 
import seaborn as sns
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter
from functions import prepare_and_model_data,extract_selected_variables
from functions import calculate_region_bounds,filter_dates,retrieve_mx2t6_mn2t6,retrieve_tp_10_u_10v_ssr,retrieve_2d_2t
from datetime import datetime, timedelta,date
from sklearn.preprocessing import StandardScaler
import geopandas as gpd
import ast
root_directory = os.getcwd()[0:3]

# Variables that need modification
crop = '02_Wheat';countryID ='06_India';country =countryID.split('_')[1]
region ='I';shp_name = 'Indiagee_up_load_winter.shp'
inpath_dates_other = root_directory + '\\SCI\\SCI9_1\\02_data\\'+crop+'\\'+countryID+'\\'+'01_data'+'\\'+'07_Information'
other_infornamtion = pd.read_csv(os.path.join(inpath_dates_other,'information.txt'), sep=' ', header=None)
startyear,endyear,shp_name = other_infornamtion.iloc[0,0],other_infornamtion.iloc[0,1],other_infornamtion.iloc[0,2]

regions = ['I']
Forecastyears = {'I': endyear}


# Directory change
inputpath_base = root_directory + '\\SCI\\SCI9_1\\02_data\\'+crop+'\\'+countryID+'\\'
yield_type = 'actual_yield';

############## Regional settings #############################################
shp_all = os.path.join(inputpath_base,'01_data','02_shp',shp_name)
gdf_all = gpd.read_file(shp_all)



# Read and extract basic growing season information
inpath_dates = os.path.join(inputpath_base, '01_data','05_buildmodel', '02_extractdates','gs_three_periods.txt')
gs_infornamtion = pd.read_csv(inpath_dates, delim_whitespace=True, header=None)
gs_infornamtion.columns = ['start_point', 'peak', 'harvest_point', 'VI_select2','regionID']

In [ ]:
# Also modified to reduce rolling weeks - only need to download the current week's data
def find_weeks(inputpath_base, forecastDataList):
    
    # Read the corresponding weeks
    file_path = os.path.join(inputpath_base, '02_S2S', '01_dataori', 'ECMWF', 'CommonYear_Week.txt')
    with open(file_path, 'r') as file:
        week_dates = [line.strip() for line in file.readlines()]
    result = []
    
    # Iterate through each date in forecastDataList
    for date in forecastDataList:
        # Iterate through week_dates to find the week corresponding to the date
        for i in range(len(week_dates) - 1):
            # Check if the date is within the current date range (inclusive of lower bound, exclusive of upper bound)
            if week_dates[i] <= date < week_dates[i + 1]:
                result.append((date, i + 1))  # Week 1 corresponds to index 0, so week number is i + 1
                break
        # Handle dates beyond the last date range (i.e., the range for Week 46)
        else:
            if date >= week_dates[-1]:
                result.append((date, len(week_dates)))  # Last week: Week 46
    result = {date: week for date, week in result}
    return result

# Download S2S dataset from ECMWF

# Also modified to reduce rolling weeks - only need to download the current week's data
origen = "ecmf"
institution = 'ECMWF'  # Data source
hyear = '2021'


for region in regions:
    # Filter the shapefile for the sub-region and get spatial extent
    if 'region' not in gdf_all.columns:
        gdf_all['region'] = 'I'
        gdf_all_sel = gdf_all
    else:
        gdf_all_sel = gdf_all  # [gdf_all['region']==region] # [ToDo] Verify if there are sub-regions
    region_limits = calculate_region_bounds(gdf_all_sel, offset=2)
    
    # Filter backward dates
    start_point, peak, harvest_point, VI_select2, region = gs_infornamtion[gs_infornamtion['regionID'] == region].iloc[0]
    # Note: Aggregation to 8-day periods is backward-looking. For example, "01-01" refers to aggregated indicators from 01-01 to 01-08

    ############## Filter backward dates #######################################################
    if start_point < harvest_point:  # Growing season in the same year
        weeks = len(list(range(start_point, harvest_point)))
    else:
        weeks = len(list(range(1, harvest_point))) + len(list(range(start_point, 46 + 1)))
    
    filtered_dates = filter_dates(inputpath_base, hyear, harvest_point, institution, weeks)
    # weeks represents the number of 8-day weeks in the growing season

    ########################### Simplify forecast data download #####################
    ######## If there are 2-3 downloads per week, select only one
    new_dates = [date[5:] for date in filtered_dates]  # Extract "mm-dd" from "yyyy-mm-dd" format
    dataweeks = find_weeks(inputpath_base, new_dates)
    
    # Keep only one date per unique week
    unique_values = {}
    for key, value in dataweeks.items():
        if value not in unique_values.values():
            unique_values[key] = value
    keys_list = list(unique_values.keys())
    filtered_dates = [hyear + '-' + date for date in keys_list]  # Restore to "yyyy-mm-dd" format
    
    Forecastyear = Forecastyears[region]    
    # Ensure data is downloaded for the forecast year
    for date in filtered_dates:
        years = str(Forecastyear) + '_' + str(Forecastyear)
        # Modify the study period here (startyear and endyear) if needed
        hdates = '/'.join([f'{week}{date[4:]}' for week in range(endyear, endyear + 1)])
        
        # Download mx2t6 (maximum 2m temperature 6-hourly)
        parameters = '121'
        variable = 'mx2t6'
        retrieve_mx2t6_mn2t6(date, parameters, variable, hdates, inputpath_base, origen, step1, num, region_limits, region, country, institution, years)
        
        # Download mn2t6 (minimum 2m temperature 6-hourly)
        parameters = '122'
        variable = 'mn2t6'
        retrieve_mx2t6_mn2t6(date, parameters, variable, hdates, inputpath_base, origen, step1, num, region_limits, region, country, institution, years)
        
        # Download tp (total precipitation)
        parameters = '228228'
        variable = 'tp'
        retrieve_tp_10_u_10v_ssr(date, parameters, variable, hdates, inputpath_base, origen, step2, num, region_limits, region, country, institution, years)
        
        # Download ssr (surface solar radiation)
        parameters = '176'
        variable = 'ssr'
        retrieve_tp_10_u_10v_ssr(date, parameters, variable, hdates, inputpath_base, origen, step3, num, region_limits, region, country, institution, years)
        
        # Download 10u (10m zonal wind)
        parameters = '165'
        variable = '10u'
        retrieve_tp_10_u_10v_ssr(date, parameters, variable, hdates, inputpath_base, origen, step3, num, region_limits, region, country, institution, years)
        
        # Download 10v (10m meridional wind)
        parameters = '166'
        variable = '10v'
        retrieve_tp_10_u_10v_ssr(date, parameters, variable, hdates, inputpath_base, origen, step3, num, region_limits, region, country, institution, years)
        
        # Download 2d (2 metre dewpoint temperature)
        parameters = '168'
        variable = '2d'
        retrieve_2d_2t(date, parameters, variable, hdates, inputpath_base, origen, step4, num, region_limits, region, country, institution, years)      
        
        # Download 2t (2 metre temperature)
        parameters = '167'
        variable = '2t'
        retrieve_2d_2t(date, parameters, variable, hdates, inputpath_base, origen, step4, num, region_limits, region, country, institution, years)

In [ ]:
# When harvest_point is 46 (corresponding to December 27th), we can only go up to December 31st at most. Therefore, the last week is less than 8 days, with a maximum of 5 days.